In [1]:
import csv
import numpy as np
X = []
with open("./data/train/crawler/data/tweets.txt.text", newline='', encoding='utf8') as file_data:
    i = 0 
    for row in file_data:
        if i < 500:
            i = i +1
            X.append(row)
            
y = []
with open("./data/train/crawler/data/tweets.txt.labels", newline='', encoding='utf8') as file_data:
    j = 0
    for row in file_data:
        if j < 500:
            j = j +1
            y.append(row.replace("\n",""))


In [2]:
X = np.array(X)
y = np.array(y)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_dtm = vect.fit_transform(X)

In [4]:
test=[]
with open("./data/test/us_test.text", newline='', encoding='utf8') as test_data:
    file = test_data.readlines()
    for row in file:
        test.append(row.replace("\n",""))

test = np.asarray(test)
test_dtm = vect.transform(test)

In [5]:
test_label = []
with open("./data/test/us_test.labels", newline='', encoding='utf8') as test_data_label:
    file = test_data_label.readlines()
    for row in file:
        test_label.append(row.replace("\n",""))
test_label = np.asarray(test_label)
test_label = test_label.reshape(-1,1)

In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

models = [
    #RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    #RandomForestClassifier(n_estimators=200, random_state=0, criterion='entropy'),
    RandomForestClassifier(n_estimators=200)    
]

CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []

for model in models:
    model_name = model.__class__.__name__
    model.fit(X_dtm, y)
    model.predict(test_dtm)
    score = model.score(test_dtm, test_label)
    entries.append((model_name, score))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'score'])

In [10]:
print (cv_df)

               model_name    score
0  RandomForestClassifier  0.23544


In [48]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

# Gini works better than entropy and more estimators, better the score is
parameters = {'n_estimators':[200, 300], 'criterion':('entropy', 'gini')}
rdf = RandomForestClassifier()

clf = GridSearchCV(rdf, parameters, cv=3, n_jobs=-1)

clf.fit(X_dtm, y)
                            
print(clf.cv_results_)

{'mean_fit_time': array([0.11071221, 0.10626809, 0.1133581 , 0.10673277, 0.11141555,
       0.10824871, 0.10428945, 0.10516866, 0.10816121, 0.10662842,
       0.10634478, 0.10543187]), 'std_fit_time': array([0.00759397, 0.00174278, 0.00459301, 0.00163899, 0.00081651,
       0.00113899, 0.00122334, 0.00122331, 0.00314407, 0.0024877 ,
       0.00162848, 0.00162532]), 'mean_score_time': array([0.00137814, 0.00140174, 0.00094024, 0.0014178 , 0.00064158,
       0.00102043, 0.00147176, 0.0007232 , 0.00146397, 0.00157015,
       0.00086896, 0.00137337]), 'std_score_time': array([5.05210801e-04, 6.37469439e-06, 3.78069242e-04, 1.78780376e-05,
       2.38680695e-05, 2.41646610e-04, 6.72843505e-04, 5.45231314e-05,
       1.27904007e-04, 4.57665162e-04, 1.72418050e-04, 1.19716413e-04]), 'param_alpha': masked_array(data=[0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.001,
                   0.001, 0.001, 0.001, 0.001, 0.001],
             mask=[False, False, False, False, False, False, False, F

/home/julien/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [13]:
print (cv_df)

               model_name    score
0      LogisticRegression  0.29528
1  RandomForestClassifier  0.21596
2               LinearSVC  0.24350
3           MultinomialNB  0.26272
